# Completion Rate - Segmentación por edad

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1) Cargar el DF
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")

# 2) Definir si se completó la sesión (llegó a 'confirm')
df["completed"] = df["process_step"] == "confirm"

# 3) Crear un DataFrame resumen a nivel de "visit_id" + "variation"
#    Para que cada sesión sea 1 fila:
df_sessions = (df
    .groupby(["visit_id", "variation"], as_index=False)
    .agg({
        "completed": "max",           # 1 si en algún momento hubo 'confirm'
        "clnt_age": "mean",           # si no varía en la misma sesión, 'mean' o 'first' da lo mismo
        "client_id": "first"          # (opcional) para saber a qué cliente está asociado
    })
)

# 4) Crear categoría de edad en este DF resumido
bins = [0, 35, 55, 120]
labels = ["<35", "35-55", ">55"]
df_sessions["age_bin"] = pd.cut(df_sessions["clnt_age"], bins=bins, labels=labels, include_lowest=True)

# 5) Chequear distribution
# print(df_sessions["age_bin"].value_counts(dropna=False))

# 6) Test de hipótesis segmentado por rango de edad
#    Para cada 'age_bin', comparamos la proporción de 'completed' en Test vs. Control
def proportion_test_for_segment(seg_df):
    cr_summary = seg_df.groupby("variation")["completed"].agg(["mean","sum","count"])
    # Chequea que existan ambos grupos
    if not {"Test","Control"}.issubset(cr_summary.index):
        return None

    test_sum = cr_summary.loc["Test","sum"]
    test_n   = cr_summary.loc["Test","count"]
    ctrl_sum = cr_summary.loc["Control","sum"]
    ctrl_n   = cr_summary.loc["Control","count"]
    
    count = np.array([test_sum, ctrl_sum])
    nobs  = np.array([test_n, ctrl_n])
    stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')
    
    return {
        "CR_Test":    cr_summary.loc["Test",    "mean"],
        "CR_Control": cr_summary.loc["Control", "mean"],
        "z_stat":     stat,
        "p_value":    p_value,
        "test_sum":   test_sum,
        "test_n":     test_n,
        "ctrl_sum":   ctrl_sum,
        "ctrl_n":     ctrl_n
    }

alpha = 0.05
age_bins = df_sessions["age_bin"].unique()

for ab in age_bins:
    if pd.isna(ab):
        # Si hay filas sin edad, se ignoran o se gestionan aparte
        continue
    
    seg_df = df_sessions[df_sessions["age_bin"] == ab]
    result = proportion_test_for_segment(seg_df)
    if result is None:
        print(f"Rango edad {ab}: No hay datos suficientes para 'Test' o 'Control'.")
        continue
    
    print(f"=== Rango de Edad: {ab} ===")
    print(f"   Completion Rate (Test)   = {result['CR_Test']:.3%}")
    print(f"   Completion Rate (Control)= {result['CR_Control']:.3%}")
    print(f"   z-stat = {result['z_stat']:.4f}, p-value = {result['p_value']:.6f}")
    
    if result["p_value"] < alpha:
        print("   -> Diferencia significativa (rechazamos H0).")
    else:
        print("   -> No hay diferencia significativa (no rechazamos H0).")
    
    print(f"   [Test completó {int(result['test_sum'])} de {int(result['test_n'])} sesiones]")
    print(f"   [Ctrl completó {int(result['ctrl_sum'])} de {int(result['ctrl_n'])} sesiones]\n")


=== Rango de Edad: >55 ===
   Completion Rate (Test)   = 53.446%
   Completion Rate (Control)= 44.406%
   z-stat = 14.1143, p-value = 0.000000
   -> Diferencia significativa (rechazamos H0).
   [Test completó 6940 de 12985 sesiones]
   [Ctrl completó 5096 de 11476 sesiones]

=== Rango de Edad: <35 ===
   Completion Rate (Test)   = 62.462%
   Completion Rate (Control)= 53.555%
   z-stat = 11.8495, p-value = 0.000000
   -> Diferencia significativa (rechazamos H0).
   [Test completó 5799 de 9284 sesiones]
   [Ctrl completó 4293 de 8016 sesiones]

=== Rango de Edad: 35-55 ===
   Completion Rate (Test)   = 59.543%
   Completion Rate (Control)= 52.921%
   z-stat = 10.1183, p-value = 0.000000
   -> Diferencia significativa (rechazamos H0).
   [Test completó 7434 de 12485 sesiones]
   [Ctrl completó 5617 de 10614 sesiones]

